In [31]:
import model
import dataloader
import importlib
importlib.reload(model)
importlib.reload(dataloader)
from model import *
from dataloader import *
import datetime
import os
import time
from tqdm import tqdm
import torch
import h5py
import numpy as np
from torchinfo import summary
import math
from torch.autograd import Variable
import pickle
# from torch.utils.data import DataLoader, Dataset


# Parameters

In [32]:
config = {
    "root" : "./data/pickles/",
    "train_feature" : "Basicmotion_feature.pickle",
    "train_target" : "Basicmotion_target.pickle",
    "test_feature" : "Basicmotion_feature_test.pickle",
    "test_target" : "Basicmotion_target_test.pickle",
    "labelencodedict" : { 'Running' : 0,
                          'Walking' : 1,
                          'Badminton' : 2,
                          'Standing' :3},
    "epochs" : 3000,
    "learning_rate" : 1e-4,
    # model 
    "first_in_channels" : 6,
    "first_out_channels" : 32,
    "first_kernel_size" : 5,
    "second_in_channels" : 32,
    "second_out_channels" : 16,
    "second_kernel_size" : 3,
    "dim_model" : 64,
    "dim_inner" : 128,
    "num_heads" : 8,
    "dropout_rate" : 0.15,
    "squeeze_factor" : 2,
    "sec_kernel_size" : 3,
    "sec_stride" : 1,
    "N" : 3,
    "gamma" : 2,
    "u" : 2,
        
}

# Get data

In [33]:
# with open(config["root"] + config["train_feature"], 'rb') as fr:
#     features = pickle.load(fr)
# with open(config["root"] + config["train_target"], 'rb') as fr2:
#     targets = pickle.load(fr2)
# with open(config["root"] + config["test_feature"], 'rb') as fr:
#     features_test = pickle.load(fr)
# with open(config["root"] + config["test_target"], 'rb') as fr2:
#     targets_test = pickle.load(fr2)

In [34]:
# class CustomDataset(Dataset):
#     def __init__(self, data, labels):
#         self.data = data
#         self.labels = labels
        
#     def __len__(self):
#         return self.data.shape[0]
    
#     def __getitem__(self, index):
#         return self.data[index], self.labels[index]

## Util Function

In [35]:
# # Util Function
# def to_categorical(y, num_classes):
#     """ 1-hot encodes a tensor """
#     a = np.eye(num_classes)[y]
#     return a

# Making Train Tensor

In [36]:
# # append features
# X_train = []
# y_train = []
# for key in features:
#     X_train.append(features[key])
#     y_train.append(targets[key])

# # make it into array
# X_train = np.array(X_train)
# y_train = np.array(y_train)
# # make y_train into 1d vector
# tmp = np.squeeze(y_train)

# # make one-hot vector
# ys = []
# for t in tmp:
#     ys.append(config['labelencodedict'][t[0]])
# train_y = to_categorical(ys, num_classes=4)

# # make X_train into tensor
# X_train_numpy = X_train.astype(np.float32)
# X_train = torch.from_numpy(X_train_numpy)
# X_train = torch.Tensor(X_train)
# # make y_train into tensor
# y_train = torch.Tensor(ys)
# # train_y is one-hot vector
# train_y = torch.from_numpy(train_y)

In [37]:
# train_dataset  = CustomDataset(X_train, y_train)
# # Define batch size
# batch_size = 8

# # Create train data loader
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# # Iterate over the data loader in your training loop
# for batch, label in train_dataloader:
#     # Perform your training operations here
#     print(batch.shape)  # Example operation, printing the batch shape
#     print(label.shape)

# Making Test Tensor

In [38]:
# # append features
# X_test = []
# y_test = []
# for key in features_test:
#     X_test.append(features_test[key])
#     y_test.append(targets_test[key])

# # make it into array
# X_test = np.array(X_test)
# y_test = np.array(y_test)

# # make y_test into 1d vector
# tmp = np.squeeze(y_test)

# # make one-hot vector
# ys_test = []
# for t in tmp:
#     ys_test.append(config['labelencodedict'][t[0]])
# test_y = to_categorical(ys_test, num_classes = len(config["labelencodedict"].keys()))


# # make X_test into tensor
# X_test_numpy = X_test.astype(np.float32)
# X_test =torch.from_numpy(X_test_numpy)
# X_test = torch.Tensor(X_test)
# # make y_test into tensor
# y_train = torch.Tensor(ys)
# # test_y is one-hot vector
# test_y = torch.Tensor(test_y)

# Functions for Train & Test

In [39]:
loss_train = []
error_train = []
def train(train_dataloader, optimizer, model = None, epochs = 30, batch_size=False, verbose = True):
    train_loss = 0.
    train_error = 0.
#     X, y = Variable(X_train), Variable(train_y)
    for epoch in range(epochs):
        epoch_loss = 0.
        epoch_error = 0.
        
        model.train()
        for batch_X, batch_y in train_dataloader:
            optimizer.zero_grad()
        
            batch_X, batch_y = Variable(batch_X), Variable(batch_y)
            print(batch_y)
            loss = model.calculate_objective(batch_X, batch_y)
            loss = loss.mean()
            error = model.calculate_classification_error(batch_X, batch_y)
            
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss#.item()
            epoch_error += error#.item()
        train_loss.append(epoch_loss / len(train_dataloader))
        train_error.append(epoch_error / len(train_dataloader))

        if verbose:
            print('Epoch: {}, Loss: {:.4f}, Train error: {:.4f}'.format(epoch, 
                                                                        epoch_loss / len(train_dataloader), 
                                                                       epoch_error / len(train_dataloader)))
    

In [40]:
# loss_train = []
# error_train = []
# def train(X_train, y_train, optimizer, model = None, epochs = 30, batch_size=False, verbose = True):
    
#     X, y = Variable(X_train), Variable(train_y)
#     for epoch in range(epochs):
#         train_loss = 0.
#         train_error = 0.
        
#         model.train()
#         for batch_X, batch_y in train_dataloader:
#             optimizer.zero_grad()

#             loss = model.calculate_objective(X, y)
#             loss = loss.mean()
#     #         print(model(X))
#             error = model.calculate_classification_error(X, y)

#             loss.backward()
#             optimizer.step()
#         if verbose:
#             print('Epoch: {}, Loss: {:.4f}, Train error: {:.4f}'.format(epoch, loss, error))


In [41]:
def test(X_test, y_test, model = None, epochs = 30, batch_size = False):
    model.eval()
    X_test, y_test = Variable(X_test), Variable(y_test)
    loss = model.calculate_objective(X_test, y_test)
    loss = loss.mean()
    error = model.calculate_classification_error(X_test, y_test)
    print('TEST, Loss: {:.4f}, Test error: {:.4f}'.format(loss, error))

# Main

In [42]:
if __name__ == "__main__":
    epochs = config["epochs"]
    learning_rate = config["learning_rate"]
#     print(X_train.shape)
    train_dataloader, test_dataloader, y_train, y_test = load_dataset(config)
    t = [i for i, _ in train_dataloader]
    ys = [j for _, j in train_dataloader]
#     print(t[0].shape[0])
    config["num_samples"] = t[0].shape[0]
    config["L"] = t[0].shape[2]
    config["length"] = t[0].shape[2]
#     print(len(config["labelencodedict"].keys()))
    md = Net(
                dim_model = config["dim_model"],
                gamma = config["gamma"],
                u = config["u"],
                ys = ys,
                first_in_channels = config["first_in_channels"],
                first_out_channels = config["first_out_channels"],
                first_kernel_size = config["first_kernel_size"],
                second_in_channels = config["second_in_channels"],
                second_out_channels = config["second_out_channels"],
                second_kernel_size = config["second_kernel_size"],
                N = config["N"],  #slice
                L = config["L"], # slice
                dim_inner = config["dim_inner"],
                num_samples = config["num_samples"], 
                length = config["length"], 
                num_heads = config["num_heads"],
                dropout_rate = config["dropout_rate"],
                squeeze_factor = config["squeeze_factor"],
                sec_kernel_size = config["sec_kernel_size"],
                sec_stride = config["sec_stride"],
                num_classes = len(config["labelencodedict"].keys()))
    # 각 layer의 이름과 파라미터 출력
    optimizer = torch.optim.Adam(md.parameters(), lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()
    
    train(train_dataloader,optimizer=optimizer, model = md, epochs = 1000, verbose = True)
    test(X_test, test_y, optimizer=optimizer, model = md, epochs = epochs )

tensor([3., 3., 2., 1., 0., 2., 0., 1.])
[tensor([ 0.1620, -0.0373,  0.0274, -0.5681,  1.8119, -0.3797, -0.8167,  1.1756,
        -1.2328,  0.3728, -0.3563,  0.7763,  1.0979,  0.5555,  0.1158,  0.2629,
         0.2655,  0.0816, -1.4559,  0.7499,  2.1749,  0.1409, -0.7384,  0.7371,
        -0.6306, -0.7719,  1.6997, -0.2863,  0.5808, -0.6369,  0.2644,  0.8801],
       grad_fn=<SliceBackward0>), tensor([-0.0051,  0.4539,  0.6353, -0.5731,  2.0595, -0.6976, -0.0543,  1.5490,
        -0.6052, -0.1935,  0.4897, -0.1000,  1.0222,  1.2183,  0.4065, -0.5550,
         0.2529,  0.5205, -1.6043,  0.6465,  1.9033,  0.6995, -0.0164,  1.0247,
        -0.3869, -0.7728,  1.4752, -0.4032,  0.3569, -0.7497, -0.3159,  0.4214],
       grad_fn=<SliceBackward0>), tensor([ 0.1294, -0.2751, -0.1461, -0.1087,  1.4161, -0.0497, -0.2308,  0.5165,
        -0.4146,  0.0903,  0.2212,  0.7531,  0.1108, -0.0660,  0.2465, -0.0452,
         0.4677,  0.4760, -1.1353,  0.3975,  1.3376,  0.0876, -0.3958,  0.6081,
        

C:\Users\yuins\Hanyang_23_1st\AI\model.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_prob = torch.nn.functional.softmax(scaled_att_score)
C:\Users\yuins\Hanyang_23_1st\AI\model.py:239: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attention_prob = torch.nn.functional.softmax(scaled_att_score)


[tensor([ 0.2378, -0.0689, -0.0974, -0.3835,  1.5874, -0.1456, -0.8418,  1.2218,
        -1.2388,  0.2855, -0.1981,  0.7416,  1.1008,  0.4847,  0.2301,  0.3336,
         0.2642,  0.2134, -1.2756,  0.5289,  2.1787,  0.1405, -0.6115,  0.5510,
        -0.7693, -1.0135,  1.6090, -0.2465,  0.7359, -0.6369,  0.4104,  0.7484],
       grad_fn=<SliceBackward0>), tensor([-0.5080,  1.5069, -0.1401, -0.8765,  2.0511, -0.2084, -0.0749,  2.0198,
        -0.7203, -0.3860, -1.0875,  0.4041,  0.6194,  1.7786,  0.6624, -0.0606,
        -0.7693, -0.8917, -2.8915,  1.2761,  3.2315,  0.6478,  0.1889,  1.6784,
        -1.4739,  0.3298,  2.7382, -0.3977,  0.6368, -0.2018, -1.8448,  0.7518],
       grad_fn=<SliceBackward0>), tensor([-0.2768,  0.6531, -0.2212, -0.5221,  2.3583, -0.7925, -0.4083,  1.6544,
        -0.8912, -1.1253,  0.4935,  0.2246,  0.1838,  0.4854,  0.7654, -0.6438,
        -0.1950, -0.4222, -2.2002,  0.5035,  2.4468,  0.5153,  0.0598,  1.7767,
        -1.3179,  0.4366,  1.1945, -0.8785,  0.43

IndexError: list index out of range

In [ ]:
[i for i in train_dataloader]